# Neighbour checks for quality control flags
Covers QC16-25

## Table of contents
[Neighbourhood functions](#neighbourhood-functions)  
[QC16 Daily neighbours (wet)](#QC16---Daily-neighbours-wet)  
[QC17 Hourly neighbours (wet)](#QC17---Hourly-neighbours-wet)  
[QC18 Daily neighbours (dry)](#QC18---Daily-neighbours-dry)  
[QC19 Hourly neighbours (dry)](#QC19---Hourly-neighbours-dry)  
[QC20 Monthly neighbours](#QC20---Monthly-neighbours)  
[QC21 Timing offset](#QC21---Timing-offset)  
[QC22 Pre-QC Affinity](#QC22---Pre-QC-Affinity)  
[QC23 Pre-QC Pearson](#QC23---Pre-QC-Pearson)  
[QC24 Daily factor](#QC24---Daily-factor)  
[QC25 Monthly factor](#QC25---Monthly-factor)  

See '3.3.4 Neighbouring gauge checks on large values' in Lewis et al. (2021)

In [2]:
import datetime
import glob

import zipfile  ## used once for Intense format data
import pandas as pd  ## used once for Intense format data

import polars as pl
import numpy as np

import scipy.stats
import geopy.distance

In [3]:
TARGET_STATION_ID = "DE_02483"
DISTANCE_THRESHOLD = 50  # 50 km
OVERLAP_THRESHOLD = 365 * 3  # three years

## Data reading globals
GAUGE_DATA_PATH = "../data/gauge_data"
DATA_ROWS_TO_SKIP = 20  ## First 20 rows are metadata TODO: what if not?
UNIT_COL = "new_units"  ## There is an original_units col too TODO: think of way to do this for different data

MULTIPLYING_FACTORS = {"hourly": 24, "daily": 1}  ## compared to daily reference

In [4]:
def read_metadata(data_path):
    metadata = {}

    with open(data_path, "r") as f:
        while True:
            key, val = f.readline().strip().split(":", maxsplit=1)
            key = key.lower().replace(" ", "_")
            metadata[key.strip()] = val.strip()
            if key == "other":
                break
    return metadata

In [5]:
station_metadata = read_metadata(data_path="../data/gauge_data/DE_02483.txt")
station_metadata["start_datetime"] = datetime.datetime.strptime(
    station_metadata["start_datetime"], "%Y%m%d%H"
)
station_metadata["end_datetime"] = datetime.datetime.strptime(
    station_metadata["end_datetime"], "%Y%m%d%H"
)

In [6]:
def add_datetime_to_gauge_data(station_metadata, gauge_data, time_multiplying_factor):
    """
    Add datetime column to gauge data using metadata from that gauge.
    NOTE: Could maybe extend so can find metadata if not provided?
    """
    startdate = station_metadata["start_datetime"]
    enddate = station_metadata["end_datetime"]
    assert isinstance(
        startdate, datetime.datetime
    ), "Please convert start_ and end_datetime to datetime.datetime objects"

    date_interval = []
    delta_days = (enddate + datetime.timedelta(days=1) - startdate).days
    for i in range(delta_days * time_multiplying_factor):
        date_interval.append(startdate + datetime.timedelta(hours=i))

    ## add datetime column
    assert len(gauge_data) == len(date_interval)
    gauge_data = gauge_data.with_columns(
        time=pl.Series(date_interval)
    )  ## set time columns

    return gauge_data

In [7]:
rain_col = f"rain_{station_metadata['original_units']}"

In [8]:
## read in gauge data
target_gauge = pl.read_csv(
    "../data/gauge_data/DE_02483.txt",
    skip_rows=20,
    schema_overrides={rain_col: pl.Float64},
)

In [9]:
target_gauge = add_datetime_to_gauge_data(
    station_metadata,
    target_gauge,
    time_multiplying_factor=MULTIPLYING_FACTORS["hourly"],
)
target_gauge = target_gauge.select(["time", rain_col])  ## Reorder (to look nice)

In [10]:
## make no data vals nans
target_gauge = target_gauge.with_columns(
    pl.when(pl.col(rain_col) == int(station_metadata["no_data_value"]))
    .then(np.nan)
    .otherwise(pl.col(rain_col))
    .alias(rain_col)
)

In [11]:
target_gauge.head()

time,rain_mm
datetime[μs],f64
2006-01-01 00:00:00,0.9
2006-01-01 01:00:00,0.3
2006-01-01 02:00:00,0.3
2006-01-01 03:00:00,0.0
2006-01-01 04:00:00,0.0


# Neighbourhood functions
TODO: convert to Classes

### Part 1. Make or read summary metadata of stations

In [12]:
## Could work by checking if metadata already exists (or user can input)
all_gauge_data_paths = glob.glob(f"{GAUGE_DATA_PATH}/*.txt")
all_gauge_data_paths

['../data/gauge_data/DE_06303.txt',
 '../data/gauge_data/DE_02718.txt',
 '../data/gauge_data/DE_04313.txt',
 '../data/gauge_data/DE_00390.txt',
 '../data/gauge_data/DE_02483.txt',
 '../data/gauge_data/DE_03215.txt',
 '../data/gauge_data/DE_00389.txt',
 '../data/gauge_data/DE_06264.txt',
 '../data/gauge_data/DE_01300.txt',
 '../data/gauge_data/DE_04488.txt',
 '../data/gauge_data/DE_00310.txt']

In [13]:
all_station_metadata_list = []
for ind, file in enumerate(all_gauge_data_paths):
    one_station_metadata = read_metadata(data_path=file)
    all_station_metadata_list.append(one_station_metadata)

In [14]:
all_station_metadata = pl.from_dicts(all_station_metadata_list)
all_station_metadata = all_station_metadata.with_columns(
    pl.col("latitude").cast(pl.Float64),
    pl.col("longitude").cast(pl.Float64),
    (pl.col("start_datetime") + "00").str.strptime(pl.Datetime, "%Y%m%d%H%M"),
    (pl.col("end_datetime") + "00").str.strptime(pl.Datetime, "%Y%m%d%H%M"),
)
all_station_metadata.head()

station_id,country,original_station_number,original_station_name,path_to_original_data,latitude,longitude,start_datetime,end_datetime,elevation,number_of_records,percent_missing_data,original_timestep,new_timestep,original_units,new_units,time_zone,daylight_saving_info,no_data_value,resolution,other
str,str,str,str,str,f64,f64,datetime[μs],datetime[μs],str,str,str,str,str,str,str,str,str,str,str,str
"""DE_06303""","""Germany""","""06303""","""NA""","""B:/INTENSE data/Original data/…",51.2884,8.5907,2006-01-01 00:00:00,2010-12-31 23:00:00,"""588m""","""43824""","""0.00""","""1hr""","""1hr""","""mm""","""mm""","""CET""","""NA""","""-999""","""0.10""",""""""
"""DE_02718""","""Germany""","""02718""","""NA""","""B:/INTENSE data/Original data/…",51.288,8.7928,2006-01-01 00:00:00,2010-12-31 23:00:00,"""458m""","""43824""","""0.00""","""1hr""","""1hr""","""mm""","""mm""","""CET""","""NA""","""-999""","""0.10""",""""""
"""DE_04313""","""Germany""","""04313""","""NA""","""B:/INTENSE data/Original data/…",51.4966,8.4342,2006-01-01 00:00:00,2010-12-31 23:00:00,"""361m""","""43824""","""0.00""","""1hr""","""1hr""","""mm""","""mm""","""CET""","""NA""","""-999""","""0.10""",""""""
"""DE_00390""","""Germany""","""00390""","""NA""","""B:/INTENSE data/Original data/…",50.9837,8.3679,2006-01-01 00:00:00,2010-12-31 23:00:00,"""610m""","""43824""","""0.00""","""1hr""","""1hr""","""mm""","""mm""","""CET""","""NA""","""-999""","""0.10""",""""""
"""DE_02483""","""Germany""","""02483""","""NA""","""B:/INTENSE data/Original data/…",51.1803,8.4891,2006-01-01 00:00:00,2010-12-31 23:00:00,"""839m""","""43824""","""0.00""","""1hr""","""1hr""","""mm""","""mm""","""CET""","""NA""","""-999""","""0.10""",""""""


### Part 2. Compute distance from target station
TODO: What if the location data is in a different projection i.e. EPSG: 27700?  
TODO: Remove duplicates

In [15]:
def compute_distance_from_target_id(metadata, target_id):
    target_station = metadata.filter(pl.col("station_id") == target_id)
    target_latlon = (
        target_station["latitude"].item(),
        target_station["longitude"].item(),
    )

    neighbour_distances = {}
    for other_station_id, other_lat, other_lon in metadata[
        ["station_id", "latitude", "longitude"]
    ].rows():

        neighbour_distances[other_station_id] = geopy.distance.geodesic(
            target_latlon, (other_lat, other_lon)
        ).kilometers
    return neighbour_distances

In [16]:
neighbours_distances = compute_distance_from_target_id(
    all_station_metadata, TARGET_STATION_ID
)
neighbours_distances

{'DE_06303': 13.96385857171046,
 'DE_02718': 24.361752991036987,
 'DE_04313': 35.397225659812456,
 'DE_00390': 23.462778369145934,
 'DE_02483': 0.0,
 'DE_03215': 15.710073576478786,
 'DE_00389': 18.844342416100808,
 'DE_06264': 28.343769567230837,
 'DE_01300': 24.64263807685722,
 'DE_04488': 15.929311137997068,
 'DE_00310': 13.134594190689885}

In [17]:
# ALTERNATIVE: do we want to avoid using geopy.distance and simply write a distance function?
# # ALTERNATIVE: maybe precompute a matrix of distances?
# all_station_dist_mtx = scipy.spatial.distance.cdist(all_station_metadata[['latitude', 'longitude']].rows(),
#                                         all_station_metadata[['latitude', 'longitude']].rows(),
#                                         metric=lambda pnt1, pnt2: geopy.distance.geodesic(pnt1, pnt2).kilometers)

In [18]:
neighbours_distances_df = pl.DataFrame(
    {
        "station_id": neighbours_distances.keys(),
        "distances": neighbours_distances.values(),
    }
)
neighbours_distances_df

station_id,distances
str,f64
"""DE_06303""",13.963859
"""DE_02718""",24.361753
"""DE_04313""",35.397226
"""DE_00390""",23.462778
"""DE_02483""",0.0
…,…
"""DE_00389""",18.844342
"""DE_06264""",28.34377
"""DE_01300""",24.642638


In [19]:
## Subset based on 50 km
close_neighbours = neighbours_distances_df.filter(
    (pl.col("distances") <= DISTANCE_THRESHOLD) & (pl.col("distances") != 0)
)

## closest 10
close_neighbours.sort("distances")[:10]

station_id,distances
str,f64
"""DE_00310""",13.134594
"""DE_06303""",13.963859
"""DE_03215""",15.710074
"""DE_04488""",15.929311
"""DE_00389""",18.844342
"""DE_00390""",23.462778
"""DE_02718""",24.361753
"""DE_01300""",24.642638
"""DE_06264""",28.34377


### Part 3. Compute the temporal overlap

In [20]:
def compute_overlap_days(start_1, end_1, start_2, end_2):
    ## TODO: add cast to datetime functionality/checks
    ## compute overlap
    overlap_start = max(start_1, start_2)
    overlap_end = min(end_1, end_2)

    overlap_days = max(0, (overlap_end - overlap_start).days)

    return overlap_days

In [21]:
def compute_overlap_days_from_target_id(metadata, target_id):
    target_station = metadata.filter(pl.col("station_id") == target_id)
    start_1, end_1 = (
        target_station["start_datetime"].item(),
        target_station["end_datetime"].item(),
    )

    neighbour_overlap_days = {}
    for other_station_id, start_2, end_2 in metadata[
        ["station_id", "start_datetime", "end_datetime"]
    ].rows():
        if target_id == other_station_id:
            continue

        neighbour_overlap_days[other_station_id] = compute_overlap_days(
            start_1, end_1, start_2, end_2
        )
    return neighbour_overlap_days

In [22]:
neighbour_overlap_days = compute_overlap_days_from_target_id(
    all_station_metadata, TARGET_STATION_ID
)
neighbour_overlap_days

{'DE_06303': 1825,
 'DE_02718': 1825,
 'DE_04313': 1825,
 'DE_00390': 1825,
 'DE_03215': 1309,
 'DE_00389': 425,
 'DE_06264': 1825,
 'DE_01300': 1825,
 'DE_04488': 1613,
 'DE_00310': 1825}

#### Subset based on 3 years

In [23]:
neighbour_overlap_days_df = pl.DataFrame(
    {
        "station_id": neighbour_overlap_days.keys(),
        "overlap_days": neighbour_overlap_days.values(),
    }
)
neighbour_overlap_days_df

station_id,overlap_days
str,i64
"""DE_06303""",1825
"""DE_02718""",1825
"""DE_04313""",1825
"""DE_00390""",1825
"""DE_03215""",1309
"""DE_00389""",425
"""DE_06264""",1825
"""DE_01300""",1825
"""DE_04488""",1613


In [24]:
neighbour_overlap_days_df.filter(pl.col("overlap_days") >= OVERLAP_THRESHOLD)

station_id,overlap_days
str,i64
"""DE_06303""",1825
"""DE_02718""",1825
"""DE_04313""",1825
"""DE_00390""",1825
"""DE_03215""",1309
"""DE_06264""",1825
"""DE_01300""",1825
"""DE_04488""",1613
"""DE_00310""",1825


## Part 4. Bring together to get neighbours both close and overlapping

In [25]:
num_closest_gauges = 10  ## based on IntenseQC

In [26]:
## Subset based on 50 km
close_neighbour_ids = neighbours_distances_df.filter(
    (pl.col("distances") <= DISTANCE_THRESHOLD) & (pl.col("distances") != 0)
)
## closest 10 values
closest_neighbour_ids = close_neighbour_ids.sort("distances")[:num_closest_gauges][
    "station_id"
].to_list()

## Subset based on 3 years
overlapping_neighbour_ids = neighbour_overlap_days_df.filter(
    pl.col("overlap_days") >= OVERLAP_THRESHOLD
)["station_id"].to_list()

In [27]:
all_neighbour_ids = set(overlapping_neighbour_ids).intersection(
    set(closest_neighbour_ids)
)
all_neighbour_ids

{'DE_00310',
 'DE_00390',
 'DE_01300',
 'DE_02718',
 'DE_03215',
 'DE_04313',
 'DE_04488',
 'DE_06264',
 'DE_06303'}

In [28]:
all_neighbour_ids_paths = {}
for id in all_neighbour_ids:
    ids_path = glob.glob(f"{GAUGE_DATA_PATH}/*{id}.txt")
    assert len(ids_path) == 1, f"There are {len(ids_path)} data files for {id}"
    all_neighbour_ids_paths[id] = ids_path[0]

In [29]:
all_neighbour_ids_paths

{'DE_06264': '../data/gauge_data/DE_06264.txt',
 'DE_00310': '../data/gauge_data/DE_00310.txt',
 'DE_04488': '../data/gauge_data/DE_04488.txt',
 'DE_04313': '../data/gauge_data/DE_04313.txt',
 'DE_03215': '../data/gauge_data/DE_03215.txt',
 'DE_02718': '../data/gauge_data/DE_02718.txt',
 'DE_00390': '../data/gauge_data/DE_00390.txt',
 'DE_01300': '../data/gauge_data/DE_01300.txt',
 'DE_06303': '../data/gauge_data/DE_06303.txt'}

## Part 5. Get neighbouring GDSR gauge by ID (an example)

In [30]:
def get_neighbour_gauge_data(neighbour_gauge_id, time_multiplying_factor):
    data_path = all_neighbour_ids_paths[neighbour_gauge_id]
    station_metadata = all_station_metadata.filter(
        pl.col("station_id") == neighbour_gauge_id
    )
    assert (
        len(station_metadata) == 1
    ), f"There are {len(station_metadata)} metadata values for {neighbour_gauge_id}. Investigate because there should only be one"
    station_metadata = station_metadata.to_dicts()[0]  ## convert df to a dict

    ## Read in gauge data
    units = station_metadata[UNIT_COL]
    rain_col = f"rain_{units}"
    gauge_data = pl.read_csv(
        data_path, skip_rows=DATA_ROWS_TO_SKIP, schema_overrides={rain_col: pl.Float64}
    )

    ## make datetime column
    gauge_data_w_dates = add_datetime_to_gauge_data(
        station_metadata, gauge_data, time_multiplying_factor=time_multiplying_factor
    )
    gauge_data_w_dates = gauge_data_w_dates.select(
        ["time", rain_col]
    )  ## Reorder (to look nice)

    return gauge_data_w_dates

In [31]:
get_neighbour_gauge_data(
    neighbour_gauge_id="DE_06264", time_multiplying_factor=MULTIPLYING_FACTORS["hourly"]
)

time,rain_mm
datetime[μs],f64
2006-01-01 00:00:00,0.0
2006-01-01 01:00:00,0.1
2006-01-01 02:00:00,0.0
2006-01-01 03:00:00,0.0
2006-01-01 04:00:00,0.0
…,…
2010-12-31 19:00:00,0.0
2010-12-31 20:00:00,0.0
2010-12-31 21:00:00,0.0


## Part 6. Get neighbouring GPCC gauge by ID (an example)

#### Note:
In the original methodology, GPCC is extracted on the fly

Hence, this needs to be refactored

In [32]:
existing_gpcc_daily_paths = {}
existing_gpcc_monthly_paths = {}
for neighbour_id in all_neighbour_ids_paths.keys():
    gpcc_id = neighbour_id.split("DE_")[1].lstrip("0")
    existing_gpcc_daily_paths[neighbour_id] = glob.glob(
        f"../data/GPCC/tw_{gpcc_id}.zip"
    )
    existing_gpcc_monthly_paths[neighbour_id] = glob.glob(
        f"../data/GPCC/mw_{gpcc_id}.zip"
    )

In [33]:
existing_gpcc_daily_paths

{'DE_06264': [],
 'DE_00310': ['../data/GPCC/tw_310.zip'],
 'DE_04488': [],
 'DE_04313': [],
 'DE_03215': ['../data/GPCC/tw_3215.zip'],
 'DE_02718': [],
 'DE_00390': [],
 'DE_01300': [],
 'DE_06303': ['../data/GPCC/tw_6303.zip']}

In [34]:
gpcc_id_to_use = "DE_00310"
gpcc_id_name = gpcc_id_to_use.split("DE_")[-1].lstrip("0")  # like 6303
example_dat_path = existing_gpcc_daily_paths[gpcc_id_to_use][0]
f = zipfile.ZipFile(example_dat_path).open(f"tw_{gpcc_id_name}.dat")
example_gpcc = pl.from_pandas(pd.read_csv(f, skiprows=1, header=None, sep=r"\s+"))

## drop unnecessary columns
example_gpcc = example_gpcc.drop([str(i) for i in range(4, 16)])

## make datetime column (apparently it's 7am-7pm)
example_gpcc = example_gpcc.with_columns(
    pl.datetime(pl.col("2"), pl.col("1"), pl.col("0"), 7).alias("time")
).drop(["0", "1", "2"])

## rename and reorder
example_gpcc = example_gpcc.rename({"3": rain_col})
example_gpcc = example_gpcc.select(["time", rain_col])  ## Reorder (to look nice)

example_gpcc

time,rain_mm
datetime[μs],f64
1951-01-01 07:00:00,5.2
1951-01-02 07:00:00,10.3
1951-01-03 07:00:00,0.0
1951-01-04 07:00:00,3.7
1951-01-05 07:00:00,3.5
…,…
2018-12-27 07:00:00,0.0
2018-12-28 07:00:00,0.2
2018-12-29 07:00:00,6.9


In [35]:
## resample into daily (also round to 1 decimal place) TODO: check offset='7h' part
target_gauge_daily = (
    target_gauge.group_by_dynamic("time", every="1d", offset="7h")
    .agg(
        [
            pl.len().alias("n_hours"),
            pl.col(rain_col).mean().round(1).alias(rain_col),
        ]
    )
    .filter(pl.col("n_hours") == 24)
    .drop("n_hours")
)  # Ensure at least 24 data points
target_gauge_daily

time,rain_mm
datetime[μs],f64
2006-01-01 07:00:00,0.2
2006-01-02 07:00:00,0.0
2006-01-03 07:00:00,0.0
2006-01-04 07:00:00,0.0
2006-01-05 07:00:00,0.0
…,…
2010-12-26 07:00:00,0.2
2010-12-27 07:00:00,0.0
2010-12-28 07:00:00,0.0


In [36]:
joined_gauges_gpcc = target_gauge_daily.join(
    example_gpcc, on="time", suffix=f"_GPCC_{gpcc_id_name}"
)
joined_gauges_gpcc = joined_gauges_gpcc.drop_nans()
joined_gauges_gpcc.head()

time,rain_mm,rain_mm_GPCC_310
datetime[μs],f64,f64
2006-01-01 07:00:00,0.2,0.2
2006-01-02 07:00:00,0.0,0.0
2006-01-03 07:00:00,0.0,0.0
2006-01-04 07:00:00,0.0,0.0
2006-01-05 07:00:00,0.0,0.0


## Step 7 Compute factor, affinity index and correlation 

In [37]:
a = np.around(joined_gauges_gpcc.filter(pl.col(rain_col) >= 0.1).min()[rain_col], 1)[0]
b = np.around(
    joined_gauges_gpcc.filter(pl.col(f"{rain_col}_GPCC_{gpcc_id_name}") >= 0.1).min()[
        f"{rain_col}_GPCC_{gpcc_id_name}"
    ],
    1,
)[0]
p = max(a, b, 0.1)
print(a, b, p)

0.1 0.1 0.1


In [38]:
joined_gauges_gpcc_duplicates = joined_gauges_gpcc.with_columns(
    pl.when((pl.col(rain_col) > p) & (pl.col(f"{rain_col}_GPCC_{gpcc_id_name}") > p))
    .then(1)
    .when(
        (pl.col(rain_col) == p) & (pl.col(f"{rain_col}_GPCC_{gpcc_id_name}") == p),
    )
    .then(1)
    .when(
        (pl.col(rain_col) == p) & (pl.col(f"{rain_col}_GPCC_{gpcc_id_name}") > p),
    )
    .then(0)
    .when((pl.col(rain_col) > p) & (pl.col(f"{rain_col}_GPCC_{gpcc_id_name}") == p))
    .then(0)
    .otherwise(np.nan)
    .alias("duplicate")
)

joined_gauges_gpcc = joined_gauges_gpcc.with_columns(
    pl.when((pl.col(rain_col) > 0) & (pl.col(f"{rain_col}_GPCC_{gpcc_id_name}") > 0))
    .then(pl.col(rain_col) / pl.col(f"{rain_col}_GPCC_{gpcc_id_name}"))
    .otherwise(np.nan)
    .alias("factor")
)
joined_gauges_gpcc

time,rain_mm,rain_mm_GPCC_310,factor
datetime[μs],f64,f64,f64
2006-01-01 07:00:00,0.2,0.2,1.0
2006-01-02 07:00:00,0.0,0.0,NaN
2006-01-03 07:00:00,0.0,0.0,NaN
2006-01-04 07:00:00,0.0,0.0,NaN
2006-01-05 07:00:00,0.0,0.0,NaN
…,…,…,…
2010-12-26 07:00:00,0.2,2.9,0.068966
2010-12-27 07:00:00,0.0,0.2,NaN
2010-12-28 07:00:00,0.0,0.0,NaN


In [39]:
match = (
    joined_gauges_gpcc_duplicates["duplicate"]
    .value_counts()
    .filter(pl.col("duplicate") == 1)["count"]
    .item()
)
diff = (
    joined_gauges_gpcc_duplicates["duplicate"]
    .value_counts()
    .filter(pl.col("duplicate") == 0)["count"]
    .item()
)
perc = match / (match + diff)
p_corr = np.corrcoef(
    joined_gauges_gpcc[rain_col], joined_gauges_gpcc[f"{rain_col}_GPCC_{gpcc_id_name}"]
)[0, 1]
f_mean = joined_gauges_gpcc["factor"].drop_nans().mean()
print(f"diff: {diff}, match:{match}")
print(f"affinity: {perc}, p_corr: {p_corr}, f_mean: {f_mean}")

diff: 157, match:540
affinity: 0.7747489239598279, p_corr: 0.011471497096547798, f_mean: 0.17114986225908063


## Part 8 Compare target with neighbour (hourly and daily)
- For hourly data, the data is first converted to daily to do comparison

_Output:_ df with long list of neighbour columns and flags

Works by computing differences from target and each of its neighbours then collates all those differences and associated difference flags into a single flag/column that describes how similar target is from neighbours


In [40]:
gpcc_id_to_use = "DE_00310"
gpcc_id_name = gpcc_id_to_use.split("DE_")[-1].lstrip("0")  # like 6303

In [41]:
joined_gauges_gpcc = target_gauge_daily.join(
    example_gpcc, on="time", suffix=f"_GPCC_{gpcc_id_name}"
)
joined_gauges_gpcc = joined_gauges_gpcc.drop_nans()
joined_gauges_gpcc.head()

time,rain_mm,rain_mm_GPCC_310
datetime[μs],f64,f64
2006-01-01 07:00:00,0.2,0.2
2006-01-02 07:00:00,0.0,0.0
2006-01-03 07:00:00,0.0,0.0
2006-01-04 07:00:00,0.0,0.0
2006-01-05 07:00:00,0.0,0.0


## Part 8.1 Wet neighbours
- This is normalised difference
TODO: Problem that there is not enough data which is wetter in the target than in the GPCC neighbour

In [42]:
def normalise_data(data):
    return (data - data.min()) / (data.max() - data.min())

In [43]:
joined_gauges_gpcc_normalised_diff = joined_gauges_gpcc.with_columns(
    # get normalised difference between target and neighbour
    rain_mm_normalised_diff=normalise_data(pl.col(f"{rain_col}"))
    - normalise_data(pl.col(f"{rain_col}_GPCC_{gpcc_id_name}"))
)

In [44]:
WET_THRESHOLD = 0.3  # default in original methodology is 1
joined_gauges_gpcc_normalised_diff_filtered = joined_gauges_gpcc_normalised_diff.filter(
    (pl.col(f"{rain_col}") >= WET_THRESHOLD)
    & (pl.col(f"{rain_col}").is_finite())
    & (pl.col(f"{rain_col}_GPCC_{gpcc_id_name}").is_finite())
    & (pl.col(f"{rain_col}_normalised_diff") > 0.0)
)

In [45]:
joined_gauges_gpcc_normalised_diff_filtered

time,rain_mm,rain_mm_GPCC_310,rain_mm_normalised_diff
datetime[μs],f64,f64,f64
2006-04-25 07:00:00,0.3,0.0,0.000941
2006-05-13 07:00:00,0.3,0.0,0.000941
2006-12-23 07:00:00,112.5,0.0,0.352886
2006-12-24 07:00:00,318.8,0.0,1.0
2007-03-16 07:00:00,3.1,0.0,0.009724
…,…,…,…
2009-11-06 07:00:00,9.8,0.9,0.015486
2009-11-18 07:00:00,1.2,0.0,0.003764
2009-11-19 07:00:00,0.8,0.0,0.002509


In [46]:
if not len(joined_gauges_gpcc_normalised_diff_filtered) >= 30:
    print(
        "Original methodology needs there to be at least 30 values to fit exponential function"
    )

Original methodology needs there to be at least 30 values to fit exponential function


In [47]:
expon_params = scipy.stats.expon.fit(
    joined_gauges_gpcc_normalised_diff_filtered[f"{rain_col}_normalised_diff"]
)

In [48]:
# Calculate thresholds at key percentiles of fitted distribution
q95 = scipy.stats.expon.ppf(0.95, expon_params[0], expon_params[1])
q99 = scipy.stats.expon.ppf(0.99, expon_params[0], expon_params[1])
q999 = scipy.stats.expon.ppf(0.999, expon_params[0], expon_params[1])

q95, q99, q999

(np.float64(0.3444910378117832),
 np.float64(0.5294660929815043),
 np.float64(0.7941055682411567))

In [49]:
## Assign flags
joined_gauges_gpcc_normalised_wet_flags = (
    joined_gauges_gpcc_normalised_diff.with_columns(
        pl.when(
            (pl.col(rain_col) >= 1.0) & (pl.col(f"{rain_col}_normalised_diff") <= q95)
        )
        .then(0)
        .when(
            (pl.col(rain_col) >= 1.0)
            & (pl.col(f"{rain_col}_normalised_diff") > q95)
            & (pl.col(f"{rain_col}_normalised_diff") <= q99),
        )
        .then(1)
        .when(
            (pl.col(rain_col) >= 1.0)
            & (pl.col(f"{rain_col}_normalised_diff") > q99)
            & (pl.col(f"{rain_col}_normalised_diff") <= q999),
        )
        .then(2)
        .when((pl.col(rain_col) >= 1.0) & (pl.col(f"{rain_col}_normalised_diff") > q95))
        .then(3)
        .otherwise(0)
        .alias("wet_flags")
    )
)

In [50]:
joined_gauges_gpcc_normalised_wet_flags["wet_flags"].value_counts()

wet_flags,count
i32,u32
3,1
2,1
1,1
0,1757


## Part 8.2 Dry

In [51]:
DRY_PERIOD_DAYS = 15
gpcc_col_name = f"{rain_col}_GPCC_{gpcc_id_name}"

In [52]:
joined_gauges_gpcc_normalised_diff.head()

time,rain_mm,rain_mm_GPCC_310,rain_mm_normalised_diff
datetime[μs],f64,f64,f64
2006-01-01 07:00:00,0.2,0.2,-0.002762
2006-01-02 07:00:00,0.0,0.0,0.0
2006-01-03 07:00:00,0.0,0.0,0.0
2006-01-04 07:00:00,0.0,0.0,0.0
2006-01-05 07:00:00,0.0,0.0,0.0


In [53]:
## TODO: This might use preexisting code, as this code is essentially looking for dry periods
def convert_to_dry_spell(rain_daily, dry_period_days, col_name):
    ## Convert to polars dataframe to use 'with_columns'
    if not isinstance(rain_daily, pl.DataFrame):
        rain_daily = rain_daily.to_frame()

    ## make dry day column
    rain_daily_dry_day = add_dry_day_col(rain_daily, col_name)
    rain_daily_dry_day = rain_daily_dry_day.with_columns(
        dry_spell_fraction=pl.col("dry_day").rolling_sum(
            window_size=dry_period_days, min_samples=dry_period_days
        )
        / dry_period_days
    )
    return rain_daily_dry_day["dry_spell_fraction"]


def add_dry_day_col(rain_daily, col_name):
    rain_daily_dry_day = rain_daily.with_columns(
        (pl.col(col_name) == 0).cast(pl.Int8()).alias("dry_day")
    )
    return rain_daily_dry_day

In [54]:
joined_gauges_gpcc_normalised_diff_dry_spell = (
    joined_gauges_gpcc_normalised_diff.with_columns(
        pl.col(rain_col)
        .map_batches(
            lambda row: convert_to_dry_spell(row, DRY_PERIOD_DAYS, col_name=rain_col)
        )
        .alias("dry_spell_fraction"),
        pl.col(gpcc_col_name)
        .map_batches(
            lambda row: convert_to_dry_spell(
                row, DRY_PERIOD_DAYS, col_name=gpcc_col_name
            )
        )
        .alias(f"dry_spell_fraction_{gpcc_id}"),
    )
)

In [55]:
joined_gauges_gpcc_normalised_diff_dry_spell.head()

time,rain_mm,rain_mm_GPCC_310,rain_mm_normalised_diff,dry_spell_fraction,dry_spell_fraction_6303
datetime[μs],f64,f64,f64,f64,f64
2006-01-01 07:00:00,0.2,0.2,-0.002762,null,null
2006-01-02 07:00:00,0.0,0.0,0.0,null,null
2006-01-03 07:00:00,0.0,0.0,0.0,null,null
2006-01-04 07:00:00,0.0,0.0,0.0,null,null
2006-01-05 07:00:00,0.0,0.0,0.0,null,null


In [56]:
## TODO: why have this part?
## consider only whether dry 15-day periods are corrobated as dry by neighbours
## dry flag works on the basis of fraction of dry days within 15-day
## check based on whether 0, 1, 2 or >= 3 wet days are recorded at the neighbour when the target is dry over the 15-day period
## truncating these fractions to 2 dp below and manipulating equalities to work with these fractions, but could work in days not fractions if change the convertToDrySpell function
dry_period = DRY_PERIOD_DAYS
fraction_drydays = {}
for d in range(1, 3 + 1):
    fraction_drydays[str(d)] = np.trunc((1.0 - (float(d) / dry_period)) * 10**2) / (
        10**2
    )

fraction_drydays  # moving window, so 1 = all dry, 0 = all wet

{'1': np.float64(0.93), '2': np.float64(0.86), '3': np.float64(0.8)}

In [57]:
## Assign flags
joined_gauges_gpcc_normalised_diff_dry_flags = (
    joined_gauges_gpcc_normalised_diff_dry_spell.with_columns(
        pl.when(
            (pl.col("dry_spell_fraction") == 1.0)
            & (pl.col(f"dry_spell_fraction_{gpcc_id}") == 1.0)
        )
        .then(0)
        .when(
            (pl.col("dry_spell_fraction") == 1.0)
            & (pl.col(f"dry_spell_fraction_{gpcc_id}") < 1.0)
            & (pl.col(f"dry_spell_fraction_{gpcc_id}") >= fraction_drydays["1"]),
        )
        .then(1)
        .when(
            (pl.col("dry_spell_fraction") == 1.0)
            & (pl.col(f"dry_spell_fraction_{gpcc_id}") < fraction_drydays["1"])
            & (pl.col(f"dry_spell_fraction_{gpcc_id}") >= fraction_drydays["2"]),
        )
        .then(2)
        .when(
            (pl.col("dry_spell_fraction") == 1.0)
            & (pl.col(f"dry_spell_fraction_{gpcc_id}") < fraction_drydays["2"])
        )
        .then(3)
        .otherwise(0)
        .alias("dry_flags")
    )
)

In [58]:
joined_gauges_gpcc_normalised_diff_dry_flags["dry_flags"].value_counts()

dry_flags,count
i32,u32
2,4
3,126
1,1
0,1629


## Part 9 Compare target with neighbour (monthly) 
- Works differently from hourly and daily
- original methodology uses bfill which uses the next available value to fillNA

In [85]:
gpcc_id_to_use = "DE_03215"
gpcc_id_name = gpcc_id_to_use.split("DE_")[-1].lstrip("0")  # like 6303

In [86]:
## TODO: get gpcc monthly
joined_gauges_gpcc = target_gauge_daily.join(
    example_gpcc, on="time", suffix=f"_GPCC_{gpcc_id_name}"
)
joined_gauges_gpcc = joined_gauges_gpcc.drop_nans()
joined_gauges_gpcc.head()

time,rain_mm,rain_mm_GPCC_3215
datetime[μs],f64,f64
2006-01-01 07:00:00,0.2,0.2
2006-01-02 07:00:00,0.0,0.0
2006-01-03 07:00:00,0.0,0.0
2006-01-04 07:00:00,0.0,0.0
2006-01-05 07:00:00,0.0,0.0


In [87]:
gpcc_col_name = f"{rain_col}_GPCC_{gpcc_id_name}"

In [88]:
## resample into monthly
joined_gauges_gpcc_monthly = (
    joined_gauges_gpcc.group_by_dynamic("time", every="1mo", offset="7h")
    .agg(
        [
            pl.col(rain_col).mean().round(1).alias(rain_col),
            pl.col(gpcc_col_name).mean().round(1).alias(gpcc_col_name)
        ]
    )
)  # TODO: Ensure at least 24*n_days_in_month data points


In [89]:
joined_gauges_gpcc_monthly.head()

time,rain_mm,rain_mm_GPCC_3215
datetime[μs],f64,f64
2006-01-01 07:00:00,0.1,1.1
2006-02-01 07:00:00,0.3,3.0
2006-03-01 07:00:00,0.4,3.6
2006-04-01 07:00:00,0.3,2.3
2006-05-01 07:00:00,0.6,4.6


In [90]:
joined_gauges_gpcc_monthly_perc_diff = joined_gauges_gpcc_monthly.with_columns(
    perc_diff=((pl.col(rain_col) - pl.col(gpcc_col_name)) * 100.) / pl.col(gpcc_col_name),
    factor_diff=pl.col(rain_col) / pl.col(gpcc_col_name)
)


In [91]:
joined_gauges_gpcc_monthly_flags = joined_gauges_gpcc_monthly_perc_diff.with_columns(
    pl.when(pl.col("perc_diff") <= -100.0).then(-3)
    .when((pl.col("perc_diff") <= -50.0) & (pl.col("perc_diff") > -100.0)).then(-2)
    .when((pl.col("perc_diff") <= -25.0) & (pl.col("perc_diff") > -50.0)).then(-1)
    .when((pl.col("perc_diff") < 25.0) & (pl.col("perc_diff") > -25.0)).then(0)
    .when((pl.col("perc_diff") >= 25.0) & (pl.col("perc_diff") < 50.0)).then(1)
    .when((pl.col("perc_diff") >= 50.0) & (pl.col("perc_diff") < 100.0)).then(2)
    .when(pl.col("perc_diff") >= 100.0).then(3)
    .otherwise(None)
    .alias("precip_flags"),
    pl.when((pl.col("factor_diff") < 11) & (pl.col("factor_diff") > 9)).then(1)
    .when((pl.col("factor_diff") < 26) & (pl.col("factor_diff") > 24)).then(2)
    .when((pl.col("factor_diff") < 3) & (pl.col("factor_diff") > 2)).then(3)
    .when((pl.col("factor_diff") > 1/11) & (pl.col("factor_diff") < 1/9)).then(4)
    .when((pl.col("factor_diff") > 1/26) & (pl.col("factor_diff") < 1/24)).then(5)
    .when((pl.col("factor_diff") > 1/3) & (pl.col("factor_diff") < 1/2)).then(6)
    .otherwise(0)
    .alias("factor_flags")
)

In [92]:
joined_gauges_gpcc_monthly_flags['precip_flags'].value_counts()

precip_flags,count
i32,u32
3,2
-3,6
-2,52


In [93]:
joined_gauges_gpcc_monthly_flags['factor_flags'].value_counts()

factor_flags,count
i32,u32
4,5
0,55


In [ ]:
## Set up NaN
joined_gauges_gpcc_monthly_flags = joined_gauges_gpcc_monthly_flags.with_columns(
    pl.when(pl.col(rain_col).is_nan()).then(np.nan).otherwise(pl.col("factor_flags")).alias("factor_flags"),
    pl.when(pl.col(rain_col).is_nan()).then(np.nan).otherwise(pl.col("precip_flags")).alias("precip_flags")
)

time,rain_mm,rain_mm_GPCC_3215,perc_diff,factor_diff,precip_flags,factor_flags
datetime[μs],f64,f64,f64,f64,i32,f64
2006-01-01 07:00:00,0.1,1.1,-90.909091,0.090909,-2,0.0
2006-02-01 07:00:00,0.3,3.0,-90.0,0.1,-2,4.0
2006-03-01 07:00:00,0.4,3.6,-88.888889,0.111111,-2,0.0
2006-04-01 07:00:00,0.3,2.3,-86.956522,0.130435,-2,0.0
2006-05-01 07:00:00,0.6,4.6,-86.956522,0.130435,-2,0.0
…,…,…,…,…,…,…
2010-08-01 07:00:00,0.6,7.5,-92.0,0.08,-2,0.0
2010-09-01 07:00:00,0.0,2.8,-100.0,0.0,-3,0.0
2010-10-01 07:00:00,0.1,1.4,-92.857143,0.071429,-2,0.0


# QC16 - Daily neighbours (wet)
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- Many, although mainly this implementation opens the neighbour wet and dry functions to parameter tweaking

In [67]:
## resample data into daily (NOTE: OG method makes sure this is 7am-7pm)

In [99]:
target_gauge

time,rain_mm
datetime[μs],f64
2006-01-01 00:00:00,0.9
2006-01-01 01:00:00,0.3
2006-01-01 02:00:00,0.3
2006-01-01 03:00:00,0.0
2006-01-01 04:00:00,0.0
…,…
2010-12-31 19:00:00,0.0
2010-12-31 20:00:00,0.0
2010-12-31 21:00:00,0.0


# QC17 - Hourly neighbours (wet) 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

In [69]:
example_id = "DE_06264"

In [70]:
neighbouring_gauge = get_neighbour_gauge_data(
    neighbour_gauge_id=example_id, time_multiplying_factor=MULTIPLYING_FACTORS["hourly"]
)

In [71]:
neighbouring_gauge

time,rain_mm
datetime[μs],f64
2006-01-01 00:00:00,0.0
2006-01-01 01:00:00,0.1
2006-01-01 02:00:00,0.0
2006-01-01 03:00:00,0.0
2006-01-01 04:00:00,0.0
…,…
2010-12-31 19:00:00,0.0
2010-12-31 20:00:00,0.0
2010-12-31 21:00:00,0.0


In [72]:
for n_id in all_neighbour_ids_paths.keys():
    print(n_id)
    one_neighbouring_gauge = get_neighbour_gauge_data(
        neighbour_gauge_id=n_id, time_multiplying_factor=MULTIPLYING_FACTORS["hourly"]
    )
    joined_gauges = target_gauge.join(
        one_neighbouring_gauge, on="time", suffix=f"_{n_id}"
    )
    joined_gauges = joined_gauges.drop_nans()

    ## resample into daily (also round to 1 decimal place) TODO: remove offset part
    joined_gauges = (
        joined_gauges.group_by_dynamic("time", every="1d", offset="7h")
        .agg(
            [
                pl.len().alias("n_hours"),
                pl.col(rain_col).mean().round(1).alias(rain_col),
                pl.col(f"{rain_col}_{n_id}")
                .mean()
                .round(1)
                .alias(f"{rain_col}_{n_id}"),
            ]
        )
        .filter(pl.col("n_hours") == 24)
        .drop("n_hours")
    )  # Ensure at least 24 data points

    ## NOTE: is this necessary? Why not just read resolution of each data?
    a = np.around(joined_gauges.filter(pl.col(rain_col) >= 0.1).min()[rain_col], 1)[0]
    b = np.around(
        joined_gauges.filter(pl.col(f"{rain_col}_{n_id}") >= 0.1).min()[
            f"{rain_col}_{n_id}"
        ],
        1,
    )[0]
    p = max(a, b, 0.1)
    print(a, b, p)

    ## TODO: rename all variables
    joined_gauges_duplicates = joined_gauges.with_columns(
        pl.when((pl.col(rain_col) > p) & (pl.col(f"{rain_col}_{n_id}") > p))
        .then(1)
        .when(
            (pl.col(rain_col) == p) & (pl.col(f"{rain_col}_{n_id}") == p),
        )
        .then(1)
        .when(
            (pl.col(rain_col) == p) & (pl.col(f"{rain_col}_{n_id}") > p),
        )
        .then(0)
        .when((pl.col(rain_col) > p) & (pl.col(f"{rain_col}_{n_id}") == p))
        .then(0)
        .otherwise(np.nan)
        .alias("duplicate")
    )

    joined_gauges = joined_gauges.with_columns(
        pl.when((pl.col(rain_col) > 0) & (pl.col(f"{rain_col}_{n_id}") > 0))
        .then(pl.col(rain_col) / pl.col(f"{rain_col}_{n_id}"))
        .otherwise(np.nan)
        .alias("factor")
    )

    match = (
        joined_gauges_duplicates["duplicate"]
        .value_counts()
        .filter(pl.col("duplicate") == 1)["count"]
        .item()
    )
    diff = (
        joined_gauges_duplicates["duplicate"]
        .value_counts()
        .filter(pl.col("duplicate") == 0)["count"]
        .item()
    )
    affinity = match / (match + diff)
    p_corr = np.corrcoef(joined_gauges[rain_col], joined_gauges[f"{rain_col}_{n_id}"])[
        0, 1
    ]
    f_mean = joined_gauges["factor"].drop_nans().mean()
    print(f"diff: {diff}, match:{match}")
    print(f"affinity: {affinity}, p_corr: {p_corr}, f_mean: {f_mean}")

    print("#" * 15)

DE_06264
0.1 0.1 0.1
diff: 174, match:379
affinity: 0.6853526220614828, p_corr: 0.0024745444582926564, f_mean: 2.8794833506858817
###############
DE_00310
0.1 0.1 0.1
diff: 124, match:455
affinity: 0.7858376511226253, p_corr: 0.0049573140334253285, f_mean: 2.342682780210973
###############
DE_04488
0.1 0.1 0.1
diff: 98, match:439
affinity: 0.8175046554934823, p_corr: 0.0034253919799619027, f_mean: 2.1391302097447347
###############
DE_04313
0.1 0.1 0.1
diff: 176, match:381
affinity: 0.6840215439856373, p_corr: -0.000119383941115046, f_mean: 3.0275432253569745
###############
DE_03215
0.1 0.1 0.1
diff: 99, match:268
affinity: 0.7302452316076294, p_corr: 0.04958692601219624, f_mean: 2.738944066476523
###############
DE_02718
0.1 0.1 0.1
diff: 178, match:366
affinity: 0.6727941176470589, p_corr: 0.0036709243371909495, f_mean: 3.095154871710614
###############
DE_00390
0.1 0.1 0.1
diff: 136, match:431
affinity: 0.7601410934744268, p_corr: 0.0027833636591166303, f_mean: 2.3027668638280483
#

# QC18 - Daily neighbours (dry) 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC19 - Hourly neighbours (dry) 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC20 - Monthly neighbours
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

In [ ]:
# def check_m_neighbours(target, neighbours):
#     df = target
#     concat_list = [df]
#     ticker = 0
#     for n in neighbours:
#         dfn, dff = compare_target_to_neighbour_monthly(df, n)
#         concat_list.append(dfn)
#         if ticker == 0:
#             df_factor = dff.copy()
#         ticker += 1

#     df = pd.concat(concat_list, axis=1, join='outer')

#     cols = ["n" + str(i + 1) for i in range(len(concat_list) - 1)]
#     cols2 = ["target"]
#     cols2.extend(cols)

#     df.columns = cols2

#     df["online"] = len(concat_list) - df[cols].isnull().sum(axis=1) - 1
#     conditions = [
#         ((df[cols] == -3).T.sum() == df.online),
#         ((df[cols] <= -2).T.sum() == df.online),
#         ((df[cols] <= -1).T.sum() == df.online),
#         ((df[cols] == 3).T.sum() == df.online),
#         ((df[cols] >= 2).T.sum() == df.online),
#         ((df[cols] >= 1).T.sum() == df.online)]

#     choices = [-3, -2, -1, 3, 2, 1]

#     df["flags"] = np.select(conditions, choices, default=0)
#     df.loc[np.isnan(df['target']), 'flags'] = np.nan

#     # Add additional checks in relation to monthly neighbours climatology

#     # Calculate neighbour monthly climatology (monthly min/max across all neighbours)
#     tmp = []
#     for n in neighbours:
#         tmp.append(n)
#     df_mc = pd.concat(tmp, axis=1, join='outer')
#     cols3 = ["n" + str(i + 1) for i in range(len(tmp))]
#     df_mc.columns = cols3
#     df_mc.index = pd.to_datetime(df_mc.index, format='%Y-%m-%d')
#     df_mc_max = df_mc.groupby(df_mc.index.month).max()
#     df_mc_max['max'] = df_mc_max.max(axis=1)
#     df_mc_min = df_mc.groupby(df_mc.index.month).min()
#     df_mc_min['min'] = df_mc_min.min(axis=1)
#     df_mc2 = pd.concat(
#         [df_mc_min.drop(cols3, axis=1), df_mc_max.drop(cols3, axis=1)], axis=1,
#         join='outer')
#     df_mc2['month'] = df_mc2.index

#     # Join monthly climatology to target series
#     df['month'] = df.index.month
#     df['date'] = df.index
#     df2 = df.merge(df_mc2, left_on='month', right_on='month')
#     df2.set_index('date', inplace=True)
#     df2.sort_index(inplace=True)

#     # Adjust flag where -3 or 3 and rainfall is outside neighbours min/max range
#     df2.loc[(df2['flags'] == -3) & (df2['online'] >= 3) &
#             ((df2['target'] < (0.75 * df2['min'])) | (df2['target'] > (1.25 * df2['max']))),
#             'flags'] = -4
#     df2.loc[(df2['flags'] == 3) & (df2['online'] >= 3) &
#             ((df2['target'] < (0.75 * df2['min'])) | (df2['target'] > (1.25 * df2['max']))),
#             'flags'] = 4

#     # Flag where less than 3 neighbours but value > 2 * neighbour max or
#     # < 0.5 * neighbour min
#     df2.loc[(df2['online'] < 3) & (df2['target'] > (2.0 * df2['max'])),
#             'flags'] = 5
#     df2.loc[(df2['online'] < 3) & (df2['target'] < (0.5 * df2['min'])),
#             'flags'] = -5

#     # If less than 3 stations online than flag check as incomplete unless flag
#     # has a value of -5/5 (i.e. outside climatology range)
#     df2.loc[(df2.online < 3) & (np.absolute(df2['flags']) != 5), "flags"] = np.nan
#     dfr = df2.flags

#     return [dfr, df_factor]

# QC21 - Timing offset 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

In [73]:
# # check for and remove any duplicates in neighbours list
# neighbours = list(dict.fromkeys(neighbours))

# # get gpcc
# neighbour_dfs = []
# for idx in neighbours:
#     neighbour_start_year = self.daily_dates[self.daily_names.index(idx)][0].year
#     neighbour_end_year = self.daily_dates[self.daily_names.index(idx)][1].year
#     neighbour_dfs.append(utils.get_daily_gpcc(self.daily_path, neighbour_start_year, neighbour_end_year, idx))

# # get matching stats for nearest gauge and offset
# nearest = neighbour_dfs[0].rename(columns={"GPCC": "ts2"})
# affinity_index_lag_minus1, r2_lag_minus1, _ = utils.calculate_affinity_index_and_pearson(daily_values_lag_minus1.to_frame("ts1"), nearest)
# affinity_index_lag0, r2_lag0, factor_lag0 = utils.calculate_affinity_index_and_pearson(daily_values.to_frame("ts1"), nearest)
# affinity_index_lag_plus1, r2_lag_plus1, _ = utils.calculate_affinity_index_and_pearson(daily_values_lag_plus1.to_frame("ts1"), nearest)

# affinity_indexes = [affinity_index_lag_minus1, affinity_index_lag0, affinity_index_lag_plus1]
# r2_values = [r2_lag_minus1, r2_lag0, r2_lag_plus1]

# if affinity_indexes.index(max(affinity_indexes)) == r2_values.index(max(r2_values)):
#     offset_flag = affinity_indexes.index(max(affinity_indexes)) - 1
# else:
#     offset_flag = 0

# QC22 - Pre-QC Affinity  
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC23 - Pre-QC Pearson
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC24 - Daily factor  
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC25 - Monthly factor
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 